In [1]:
try:
    from firedrake import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    from firedrake import *

# Introduction to Firedrake using the 1D Helmholtz equation
Firedrake is a library that provides tools for solving PDEs using the finite element method. We will see some of these tools in this notebook.

The equation we will solve is the 1D Helmholtz equation
\begin{equation}
-u_{xx}+ku = f,
\end{equation}
where $k$ is a constant, $f$ is a given function and $u$ is the unknown function that we are solving for. We will solve this on the domain $0\le x\le 1$ and we will assume periodic boundary conditions. The first thing we need to do is set up a discretisation, or mesh, of this domain. We do this using one of the inbuilt Firedrake meshes. The input to this function is simply the number of cells in the domain, here 20:

In [ ]:
mesh = PeriodicUnitIntervalMesh(20)

We will now set up our right hand side function $f$ which is given by the expression
\begin{equation}
f = (1+4\pi^2)\cos(2\pi x).
\end{equation}
This expression has been chosen because we know the analytic solution for $u$ in this case (via the "method of manufactured solutions"). Before we can set up a finite element function for $f$, we have to decide what finite element function space we are going to use. Here we will use a function space consisting of continous ("CG") piecewise linear functions (the "1" indicates the order of the polynomial - in this case linear).

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
f = Function(V)

Now that we have a function, we can interpolate our chosen expression for $f$ and plot it:

In [ ]:
x = SpatialCoordinate(mesh)
f.interpolate((1+4*pi**2)*cos(2*pi*x[0]))
plot(f)

Now we need to set up the weak form of our equation, which requires us to multiply by a test function $\phi$ and integrate over the domain:
\begin{equation}
\int_0^1 (-\phi u_{xx} + k\phi u) \, dx = \int_0^1\phi f \, dx.
\end{equation}

Recall that we chose to use a piecewise linear function space and note that we have a second order derivative on the left hand side. We can reduce the differentiability requirements on our solution $u$ by integrating the second order term by parts, giving:
\begin{equation}
\int_0^1 (\phi_x u_x + k\phi u) \, dx = \int_0^1\phi f \, dx.
\end{equation}

We will take $k=1$ and set up our weak form by defining test and trial functions on our function space and writing $a$ for the left hand side and $L$ for the right hand side, we have:

In [ ]:
phi = TestFunction(V)
trial = TrialFunction(V)
a = inner(grad(phi), grad(trial))*dx + inner(phi, trial)*dx
L = inner(phi, f)*dx

We need to set up a function to store the solution and then we can solve the linear system and plot the solution.

In [ ]:
u = Function(V)
solve(a==L, u)
plot(u)

We can also set up a function and interpolate the known analytical solution to it. We can plot this and also compute an error norm to see how well our approximation has worked.

In [ ]:
u_exact = Function(V).interpolate(cos(2*pi*x[0]))
plot(u_exact)

In [ ]:
print(errornorm(u, u_exact))

## Exercises:
* Increase the resolution - try doubling the number of cells.
* Increase the order of the function space to use piecewise quadratics by changing the line defining $V$ and passing in 2 instead of 1.
* Solve this equation in 2D by changing the mesh to be PeriodicUnitSquareMesh(20, 20). If the analytical solution is
\begin{equation}
u=\cos(2\pi x)\cos(2\pi y),
\end{equation}
what does $f$ have to be for the two dimensional Helmholtz equation
\begin{equation}
-\nabla^2 u + u = f
\end{equation}
to hold?